In [31]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
import os
from tqdm import tqdm
import torch
from matplotlib import pyplot as plt
import time
import os
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, BatchSampler, RandomSampler

%matplotlib inline

In [32]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [33]:
# labels = pd.read_csv("trainingData.csv")
# le = preprocessing.LabelEncoder()
# le.fit(labels["Language"])
# labels["Language"] = le.transform(labels["Language"])
# labels.head()

In [34]:
# train_dirs = []
# train_base_dir = "Training data"
# for direc in os.listdir("Training data"):
#   train_dirs.append(direc)

In [35]:
# train_labels = []
# pbar = tqdm(total=66176)
# for i, direc in enumerate(train_dirs):
#     label = labels[labels["Sample Filename"] == direc]["Language"].item()
#     train_labels.append(label)
#     pbar.update(1)
# pbar.close()    
# labels = np.array(train_labels)    

In [36]:
# np.save("labels.npy", labels)

In [37]:
labels = np.load("labels.npy")

In [38]:
data = np.load("traindata.npy")
# np.random.shuffle(data)

In [39]:
data.shape
max(labels)

175

In [40]:
test_size = 20000
train_size = data.shape[0] - test_size
print(train_size)
train = data[:train_size, :, :]
train_label = labels[:train_size]
test = data[train_size:, :, :]
test_label = labels[train_size:]

46176


In [41]:
train.shape
train_label.shape
train_label[0]

104

In [42]:
test.shape
test_label.shape

(20000,)

In [43]:
class TrainDataset(Dataset):
  
    def __init__(self, train, label):
        self.train = train
        self.label = label

    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        train = torch.Tensor(self.train[idx])
        train = train.transpose(0,1)
#         train = train.to(DEVICE)
        label = self.label[idx]
#         label = torch.Tensor(label)
#         label = label.to(DEVICE)
        return(train, label)

In [44]:
class TestDataset(Dataset):
  
  def __init__(self, test):
    self.test = test
  
  def __len__(self):
    return self.test.shape[0]
  
  def __getitem__(self, idx):
    test = torch.Tensor(self.test[idx])
    test = test.to(DEVICE)
#     test = test.unsqueeze(0)
    return test

In [45]:
train_dataset = TrainDataset(train, train_label)
test_dataset = TestDataset(test)
train_batch_size = 100
test_batch_size = 100
train_loader = DataLoader(train_dataset,
                        batch_sampler=BatchSampler(RandomSampler(train_dataset), train_batch_size, False))
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)

In [46]:
# iter(train_loader).next()
# train_dataset[0][0].shape

In [47]:
class DetectionModel(nn.Module):
    
    def __init__(self,vocab_size,embed_size,hidden_size, nlayers):
        super(DetectionModel,self).__init__()
        self.vocab_size=vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers, bidirectional=True) # Recurrent network
        self.scoring = nn.Linear(hidden_size * 2,vocab_size) # Projection layer
        
    def forward(self, seq_batch):
        batch_size = seq_batch.size(1)
        embed = seq_batch #L x N x E
        hidden = None
        output_lstm,hidden = self.rnn(embed,hidden) #L x N x H
        output_lstm_flatten = output_lstm.view(-1,self.hidden_size * 2) #(L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x V
        return output_flatten.view(-1,batch_size,self.vocab_size)

In [48]:
def train_epoch(model, optimizer, train_loader):
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)
    batch_id=0
    for inputs,targets in train_loader:
        batch_loss = []
        batch_id+=1
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        outputs = model(inputs) # 3D
        outputs = outputs[:, -1, :] # pull out the last layer
#         print(outputs.shape)
        loss = criterion(outputs,targets) # Loss of the flattened outputs
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        torch.cuda.empty_cache()
        if batch_id % 20 == 0:
            lpw = np.mean(batch_loss)
            print("At batch",batch_id)
            print("Training loss :",lpw)

    return model
    

In [49]:
langcount = 176
model = DetectionModel(langcount,40,256,3)
model = model.to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=1e-6)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [ ]:
for i in range(50):
    train_epoch(model, optimizer, train_loader)

/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


At batch 20
Training loss : 5.17752361298
At batch 40
Training loss : 5.15928649902
At batch 60
Training loss : 5.17980051041
At batch 80
Training loss : 5.18092632294


In [19]:
torch.cuda.empty_cache()